In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from pathlib import Path
import time
import os
import datetime


In [3]:
dir = '../rawData/Foursquare_global'
sub_dir = 'city_user_day_fix'
df = pd.read_csv(f'{dir}/{sub_dir}/gb_sorted.csv') 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28299352 entries, 0 to 28299351
Data columns (total 19 columns):
 #   Column         Dtype  
---  ------         -----  
 0   user           int64  
 1   location_id    object 
 2   t              object 
 3   offset         int64  
 4   t_len          int64  
 5   check-in_time  object 
 6   day            object 
 7   p_lat          float64
 8   p_lon          float64
 9   p_type         object 
 10  p_countrycode  object 
 11  city           object 
 12  c_lat          float64
 13  c_lon          float64
 14  c_countrycode  object 
 15  c_country      object 
 16  c_type         object 
 17  hour           int64  
 18  hour_offset    int64  
dtypes: float64(4), int64(5), object(10)
memory usage: 4.0+ GB


In [ ]:
df.head(3)

In [ ]:
df['day'] = pd.to_datetime(df['day'])
df['check-in_time'] = pd.to_datetime(df['check-in_time'])
df = df.sort_values(['user', 'day', 'check-in_time']).reset_index(drop=True)
df['ori_id'] = np.arange(len(df))
ori_id = df['ori_id']

grouped = df.groupby(['user', 'day'])
gb_cnt_sorted = grouped.count().reset_index()

group_ori_id = np.concatenate([g['ori_id'].values for name, g in grouped])
np.all(group_ori_id == df['ori_id'].values)

True

In [12]:
gb_cnt_sorted['user_day_cnt'] = gb_cnt_sorted['check-in_time']
traj_id = np.concatenate([np.full((cnt,), i) for i, cnt in enumerate(gb_cnt_sorted['user_day_cnt'].tolist())])
df['traj_id'] = traj_id
len(df), len(ori_id), len(traj_id)

(28299352, 28299352, 28299352)

In [15]:
c_df = pd.read_csv("./rawData/Foursquare_global/dataset_TIST2015_Cities.txt", sep='\t', names=['city', 'c_lat', 'c_lon', 'c_countrycode', 'c_country', 'c_type'])
city_records = dict()
for city in c_df['city'].tolist():
    city_records[city] = []
    
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28299352 entries, 0 to 28299351
Data columns (total 21 columns):
 #   Column         Dtype         
---  ------         -----         
 0   user           int64         
 1   location_id    object        
 2   t              object        
 3   offset         int64         
 4   t_len          int64         
 5   check-in_time  datetime64[ns]
 6   day            datetime64[ns]
 7   p_lat          float64       
 8   p_lon          float64       
 9   p_type         object        
 10  p_countrycode  object        
 11  city           object        
 12  c_lat          float64       
 13  c_lon          float64       
 14  c_countrycode  object        
 15  c_country      object        
 16  c_type         object        
 17  hour           float64       
 18  hour_offset    float64       
 19  ori_id         int64         
 20  traj_id        int64         
dtypes: datetime64[ns](2), float64(6), int64(5), object(8)
memory usage: 4.4+ GB

In [16]:
for city in tqdm(c_df['city'].tolist()):
    city_df = df[df['city'] == city]
    city_traj_id = set(city_df['traj_id'].unique().tolist())
    city_traj_df = df[df['traj_id'].isin(city_traj_id)]
    city_traj_df.to_csv(f'{dir}/{sub_dir}/res/{city}.csv', index=False)

100%|██████████| 415/415 [14:15<00:00,  2.06s/it]
